# Pandas Exercise Solutions

This notebook contains working solutions for all the exercises.

## Setup: Load Data and Libraries

In [24]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Libraries imported successfully!")

Libraries imported successfully!


In [25]:
# Load the data files
sales = pd.read_csv('files/sales_records.csv')
products = pd.read_csv('files/products_master.csv')
regions = pd.read_csv('files/regions_dim.csv')

print(f"✓ Data loaded successfully!")
print(f"Sales: {len(sales)} rows × {len(sales.columns)} columns")
print(f"Products: {len(products)} rows × {len(products.columns)} columns")
print(f"Regions: {len(regions)} rows × {len(regions.columns)} columns")

✓ Data loaded successfully!
Sales: 260 rows × 7 columns
Products: 8 rows × 5 columns
Regions: 5 rows × 2 columns


In [26]:
# Quick preview of the data
print("Sales sample:")
display(sales.head())
print("\nProducts:")
display(products)
print("\nRegions:")
display(regions)

Sales sample:


,order_id,order_date,product_id,region_code,units,unit_price,revenue
0,SO-10000,2024-09-06,P-ENT,EMEA,6,513.77,3082.59
1,SO-10001,2024-09-02,P-ADD,APAC,18,30.91,556.34
2,SO-10002,2024-06-25,P-ANL,EMEA,8,302.84,2422.68
3,SO-10003,2025-02-20,P-PLS,EMEA,37,112.30,4155.00
4,SO-10004,2025-08-04,P-PLS,EMEA,23,108.11,2486.50



Products:


,product_id,product,category,list_price,status
0,P-BSC,Basic,Core,49.00,Active
1,P-PLS,Plus,Core,99.00,Active
2,P-PRO,Pro,Core,199.00,Active
3,P-ENT,Enterprise,Suite,499.00,Active
4,P-ANL,Analytics,Addon,299.00,Active
5,P-ADD,Add-on,Addon,29.00,Active
6,P-LGC,Legacy,Legacy,79.00,Discontinued
7,P-BTA,Beta,Beta,59.00,Preview



Regions:


,region_code,region
0,AMER,AMER
1,EMEA,EMEA
2,APAC,APAC
3,NORD,Nordics
4,ANZ,Australia/NZ


---
# Exercise 1: Basic Filtering & Boolean Logic

**Goal:** Filter sales data using multiple conditions

In [27]:
# Task 1: Sales in AMER or EMEA
task1 = sales[sales['region_code'].isin(['AMER', 'EMEA'])]
print(f"Task 1 - AMER or EMEA sales: {len(task1)} rows")

# Task 2: Plus or Pro products
task2 = sales[sales['product_id'].isin(['P-PLS', 'P-PRO'])]
print(f"Task 2 - Plus or Pro products: {len(task2)} rows")

# Task 3: 20+ units
task3 = sales[sales['units'] >= 20]
print(f"Task 3 - Transactions with 20+ units: {len(task3)} rows")

# Task 4: Combined filters
task4 = sales[
    (sales['region_code'].isin(['AMER', 'EMEA'])) & 
    (sales['product_id'].isin(['P-PLS', 'P-PRO'])) & 
    (sales['units'] >= 20)
]
print(f"Task 4 - All three filters combined: {len(task4)} rows")

print("\nSample of highly filtered results:")
display(task4[['order_id', 'product_id', 'region_code', 'units', 'revenue']].head())

Task 1 - AMER or EMEA sales: 154 rows
Task 2 - Plus or Pro products: 86 rows
Task 3 - Transactions with 20+ units: 97 rows
Task 4 - All three filters combined: 16 rows

Sample of highly filtered results:


,order_id,product_id,region_code,units,revenue
3,SO-10003,P-PLS,EMEA,37,4155.00
4,SO-10004,P-PLS,EMEA,23,2486.50
54,SO-10054,P-PLS,AMER,20,2112.26
57,SO-10057,P-PLS,AMER,31,3258.17
71,SO-10071,P-PLS,AMER,22,2290.99


In [28]:
# Bonus: Sales with >10% discount (requires joining with products first)
sales_with_products = sales.merge(products, on='product_id', how='inner')
sales_with_products['discount_pct'] = (
    (sales_with_products['list_price'] - sales_with_products['unit_price']) 
    / sales_with_products['list_price'] * 100
)
large_discounts = sales_with_products[sales_with_products['discount_pct'] > 10]

print(f"Bonus - Sales with >10% discount: {len(large_discounts)} rows")
print(f"Average discount: {large_discounts['discount_pct'].mean():.1f}%")

Bonus - Sales with >10% discount: 12 rows
Average discount: 12.2%


---
# Exercise 2: Aggregation & Grouping

**Goal:** Summarize sales by different dimensions

In [29]:
# Task 1: Revenue and units by product
by_product = sales.groupby('product_id').agg({
    'revenue': 'sum',
    'units': 'sum'
}).round(2)

print("Task 1 - Revenue and Units by Product:")
display(by_product)

Task 1 - Revenue and Units by Product:


,revenue,units
product_id,,
P-ADD,27858.94,934
P-ANL,78646.03,264
P-BSC,63236.10,1307
P-ENT,103398.34,212
P-OEM,3247.21,83
P-PLS,88102.19,879
P-PRO,93257.12,469
P-TRI,2981.93,336


In [30]:
# Task 2: Average unit price per region
avg_price_by_region = sales.groupby('region_code')['unit_price'].mean().round(2)

print("Task 2 - Average Unit Price by Region:")
display(avg_price_by_region)

Task 2 - Average Unit Price by Region:


region_code
AMER    207.48
APAC    159.32
EMEA    161.52
LATAM   254.51
NORD    174.67
Name: unit_price, dtype: float64

In [31]:
# Task 3: Product × Region summary matrix
product_region = sales.pivot_table(
    index='product_id',
    columns='region_code',
    values='revenue',
    aggfunc='sum',
    fill_value=0
).round(2)

print("Task 3 - Product × Region Revenue Matrix:")
display(product_region)

Task 3 - Product × Region Revenue Matrix:


region_code,AMER,APAC,EMEA,LATAM,NORD
product_id,,,,,
P-ADD,7157.68,6028.36,13550.26,0.00,1122.64
P-ANL,29145.76,16493.79,25082.02,3361.73,4562.73
P-BSC,23934.16,17957.99,13510.18,2113.97,5719.80
P-ENT,48927.76,29289.86,13617.57,5617.38,5945.77
P-OEM,340.29,1370.79,1536.13,0.00,0.00
P-PLS,33820.45,25607.96,23151.35,1694.17,3828.26
P-PRO,25384.96,15531.19,42964.14,1156.39,8220.44
P-TRI,0.00,1347.70,1075.55,331.91,226.77


In [32]:
# Task 4: Revenue by month
sales['order_date'] = pd.to_datetime(sales['order_date'])
sales['year_month'] = sales['order_date'].dt.to_period('M')

monthly_revenue = sales.groupby('year_month')['revenue'].sum().round(2)

print("Task 4 - Revenue by Month:")
display(monthly_revenue.head(12))

Task 4 - Revenue by Month:


year_month
2024-01   26395.92
2024-02   17673.05
2024-03   13480.63
2024-04   35497.58
2024-05   27019.00
2024-06   23233.95
2024-07   18719.39
2024-08   23298.24
2024-09   32859.73
2024-10   18942.74
2024-11   13153.44
2024-12   15079.90
Freq: M, Name: revenue, dtype: float64

In [33]:
# Task 5: Top 3 products by units in each region
top_products = (
    sales.groupby(['region_code', 'product_id'])['units']
    .sum()
    .reset_index()
    .sort_values(['region_code', 'units'], ascending=[True, False])
    .groupby('region_code')
    .head(3)
)

print("Task 5 - Top 3 Products by Units in Each Region:")
display(top_products)

Task 5 - Top 3 Products by Units in Each Region:


,region_code,product_id,units
2,AMER,P-BSC,498
5,AMER,P-PLS,339
0,AMER,P-ADD,240
9,APAC,P-BSC,367
12,APAC,P-PLS,258
7,APAC,P-ADD,195
15,EMEA,P-ADD,461
17,EMEA,P-BSC,286
20,EMEA,P-PLS,227
24,LATAM,P-BSC,43


---
# Exercise 3: Date Parsing & Time-Based Analysis

**Goal:** Work with date columns and extract time components

In [34]:
# Task 1: Parse dates (already done above)
print(f"Task 1 - Date column type: {sales['order_date'].dtype}")

# Task 2: Extract date components
sales['year'] = sales['order_date'].dt.year
sales['month'] = sales['order_date'].dt.month
sales['quarter'] = sales['order_date'].dt.quarter
sales['weekday'] = sales['order_date'].dt.day_name()

print("\nTask 2 - Date components extracted:")
display(sales[['order_date', 'year', 'month', 'quarter', 'weekday']].head())

Task 1 - Date column type: datetime64[ns]

Task 2 - Date components extracted:


,order_date,year,month,quarter,weekday
0,2024-09-06,2024,9,3,Friday
1,2024-09-02,2024,9,3,Monday
2,2024-06-25,2024,6,2,Tuesday
3,2025-02-20,2025,2,1,Thursday
4,2025-08-04,2025,8,3,Monday


In [35]:
# Task 3: Fiscal quarter (FY starts April 1)
sales['fiscal_quarter'] = ((sales['order_date'].dt.month - 4) % 12 // 3) + 1

print("Task 3 - Fiscal quarters (FY starts in April):")
display(sales[['order_date', 'quarter', 'fiscal_quarter']].head(10))

Task 3 - Fiscal quarters (FY starts in April):


,order_date,quarter,fiscal_quarter
0,2024-09-06,3,2
1,2024-09-02,3,2
2,2024-06-25,2,1
3,2025-02-20,1,4
4,2025-08-04,3,2
5,2024-06-17,2,1
6,2024-07-09,3,2
7,2024-11-07,4,3
8,2024-05-04,2,1
9,2024-12-08,4,3


In [36]:
# Task 4: Days since order
reference_date = pd.Timestamp('2025-10-22')
sales['days_since'] = (reference_date - sales['order_date']).dt.days

print("Task 4 - Most recent orders:")
display(sales.sort_values('order_date', ascending=False)[['order_date', 'days_since']].head())

Task 4 - Most recent orders:


,order_date,days_since
135,2025-10-08,14
114,2025-10-03,19
174,2025-09-27,25
26,2025-09-25,27
15,2025-09-20,32


In [37]:
# Task 5: Revenue by quarter
quarterly = sales.groupby('quarter')['revenue'].sum().round(2)

print("Task 5 - Revenue by Calendar Quarter:")
display(quarterly)

Task 5 - Revenue by Calendar Quarter:


quarter
1   112003.03
2   161679.46
3   137293.17
4    49752.20
Name: revenue, dtype: float64

---
# Exercise 4: Inner Join of Products with Sales

**Goal:** Understand inner joins and identify matched records

In [38]:
# Task 1 & 2: Inner join and count results
merged_inner = sales.merge(products, on='product_id', how='inner')

print(f"Task 1 - Inner join: {len(merged_inner)} rows (from {len(sales)} sales)")
print(f"Task 2 - Rows lost: {len(sales) - len(merged_inner)}")
print(f"Task 2 - Rows retained: {len(merged_inner)}")

Task 1 - Inner join: 242 rows (from 260 sales)
Task 2 - Rows lost: 18
Task 2 - Rows retained: 242


In [39]:
# Task 3: Find unmatched products
unmatched = set(sales['product_id']) - set(products['product_id'])

print(f"Task 3 - Products in sales but NOT in master: {unmatched}")
print("\nRevenue from these orphan products:")
orphan_sales = sales[sales['product_id'].isin(unmatched)]
print(orphan_sales[['product_id', 'revenue']].groupby('product_id')['revenue'].sum())

Task 3 - Products in sales but NOT in master: {'P-OEM', 'P-TRI'}

Revenue from these orphan products:
product_id
P-OEM   3247.21
P-TRI   2981.93
Name: revenue, dtype: float64


In [40]:
# Task 4: Show sales with product details
print("Task 4 - Sales with Product Name and Category:")
display(merged_inner[['order_id', 'product_id', 'product', 'category', 'revenue']].head())

Task 4 - Sales with Product Name and Category:


,order_id,product_id,product,category,revenue
0,SO-10000,P-ENT,Enterprise,Suite,3082.59
1,SO-10001,P-ADD,Add-on,Addon,556.34
2,SO-10002,P-ANL,Analytics,Addon,2422.68
3,SO-10003,P-PLS,Plus,Core,4155.00
4,SO-10004,P-PLS,Plus,Core,2486.50


In [41]:
# Task 5: Calculate discounts
merged_inner['discount_pct'] = (
    (merged_inner['list_price'] - merged_inner['unit_price']) 
    / merged_inner['list_price'] * 100
)

print("Task 5 - Discount Analysis:")
print(f"Average discount: {merged_inner['discount_pct'].mean():.2f}%")
print(f"Max discount: {merged_inner['discount_pct'].max():.2f}%")
print(f"Min discount: {merged_inner['discount_pct'].min():.2f}%")
print(f"Orders with actual discount (>0%): {(merged_inner['discount_pct'] > 0).sum()}")

Task 5 - Discount Analysis:
Average discount: -0.12%
Max discount: 16.68%
Min discount: -16.98%
Orders with actual discount (>0%): 114


---
# Exercise 5: Left Join - Keeping All Sales

**Goal:** Preserve all sales records even if products are unknown

In [42]:
# Task 1: Left join keeps all sales
merged_left = sales.merge(products, on='product_id', how='left')

print(f"Task 1 - Left join: {len(merged_left)} rows (all {len(sales)} sales retained)")
print(f"No data lost!")

Task 1 - Left join: 260 rows (all 260 sales retained)
No data lost!


In [43]:
# Task 2: Identify orphan sales
orphans = merged_left[merged_left['product'].isna()]

print(f"Task 2 - Orphan sales (unknown products): {len(orphans)} records")
print("\nOrphan product IDs and counts:")
display(orphans['product_id'].value_counts())

Task 2 - Orphan sales (unknown products): 18 records

Orphan product IDs and counts:


product_id
P-OEM    9
P-TRI    9
Name: count, dtype: int64

In [44]:
# Task 3: Revenue from unknown products
orphan_revenue = orphans['revenue'].sum()
total_revenue = sales['revenue'].sum()

print(f"Task 3 - Revenue from unknown products: ${orphan_revenue:,.2f}")
print(f"Total revenue: ${total_revenue:,.2f}")
print(f"Percentage of total: {orphan_revenue/total_revenue*100:.2f}%")

Task 3 - Revenue from unknown products: $6,229.14
Total revenue: $460,727.86
Percentage of total: 1.35%


In [45]:
# Task 4: Fill missing product names
merged_left['product'] = merged_left['product'].fillna('Unknown Product')
merged_left['category'] = merged_left['category'].fillna('Unknown')

print("Task 4 - Missing values filled:")
display(merged_left[merged_left['product'] == 'Unknown Product'][[
    'order_id', 'product_id', 'product', 'category', 'revenue'
]].head())

Task 4 - Missing values filled:


,order_id,product_id,product,category,revenue
16,SO-10016,P-OEM,Unknown Product,Unknown,555.74
19,SO-10019,P-OEM,Unknown Product,Unknown,363.94
58,SO-10058,P-OEM,Unknown Product,Unknown,206.22
68,SO-10068,P-OEM,Unknown Product,Unknown,353.62
76,SO-10076,P-TRI,Unknown Product,Unknown,405.00


In [46]:
# Task 5: Create orphan flag
merged_left['is_orphan'] = merged_left['list_price'].isna()

print(f"Task 5 - Orphan flag created:")
print(f"Orphan records: {merged_left['is_orphan'].sum()}")
print(f"Complete records: {(~merged_left['is_orphan']).sum()}")

Task 5 - Orphan flag created:
Orphan records: 18
Complete records: 242


---
# Exercise 6: Right Join - Finding Unsold Products

**Goal:** Identify products with no sales history

In [47]:
# Task 1: Right join keeps all products
merged_right = sales.merge(products, on='product_id', how='right')

print(f"Task 1 - Right join: {len(merged_right)} rows (all {len(products)} products included)")

Task 1 - Right join: 244 rows (all 8 products included)


In [48]:
# Task 2: Products with no sales
no_sales = merged_right[merged_right['order_id'].isna()]

print(f"Task 2 - Products with NO sales: {len(no_sales)}")
print("\nUnsold products:")
display(no_sales[['product_id', 'product', 'category', 'status', 'list_price']])

Task 2 - Products with NO sales: 2

Unsold products:


,product_id,product,category,status,list_price
242,P-LGC,Legacy,Legacy,Discontinued,79.00
243,P-BTA,Beta,Beta,Preview,59.00


In [49]:
# Task 3: Potential revenue
potential = no_sales['list_price'].sum()

print(f"Task 3 - Potential revenue from unsold products: ${potential:,.2f}")

Task 3 - Potential revenue from unsold products: $138.00


In [50]:
# Task 4: Status vs sales analysis
status_analysis = products.merge(
    sales.groupby('product_id').size().reset_index(name='sales_count'),
    on='product_id',
    how='left'
)
status_analysis['sales_count'] = status_analysis['sales_count'].fillna(0)

print("Task 4 - Sales Count by Product Status:")
display(status_analysis[['product', 'status', 'sales_count']])

print("\nProducts with zero sales:")
display(status_analysis[status_analysis['sales_count'] == 0][['product', 'status']])

Task 4 - Sales Count by Product Status:


,product,status,sales_count
0,Basic,Active,52.00
1,Plus,Active,48.00
2,Pro,Active,38.00
3,Enterprise,Active,42.00
4,Analytics,Active,33.00
5,Add-on,Active,29.00
6,Legacy,Discontinued,0.00
7,Beta,Preview,0.00



Products with zero sales:


,product,status
6,Legacy,Discontinued
7,Beta,Preview


---
# Exercise 7: Full Outer Join - Complete Picture

**Goal:** See ALL products and ALL sales, matched or not

In [51]:
# Task 1: Full outer join with indicator
merged_full = sales.merge(products, on='product_id', how='outer', indicator=True)

print(f"Task 1 - Full outer join: {len(merged_full)} rows")

Task 1 - Full outer join: 262 rows


In [52]:
# Task 2: Count by merge category
merge_counts = merged_full['_merge'].value_counts()

print("Task 2 - Merge Categories:")
display(merge_counts)
print(f"\n- Matched (both): {merge_counts.get('both', 0)} records")
print(f"- Sales orphans (left_only): {merge_counts.get('left_only', 0)} records")
print(f"- Product orphans (right_only): {merge_counts.get('right_only', 0)} records")

Task 2 - Merge Categories:


_merge
both          242
left_only      18
right_only      2
Name: count, dtype: int64


- Matched (both): 242 records
- Sales orphans (left_only): 18 records
- Product orphans (right_only): 2 records


In [53]:
# Task 3: Revenue from complete records
complete_revenue = merged_full[merged_full['_merge'] == 'both']['revenue'].sum()
total_revenue = merged_full['revenue'].sum()

print(f"Task 3 - Revenue from complete records: ${complete_revenue:,.2f}")
print(f"Total revenue: ${total_revenue:,.2f}")
print(f"Percentage: {complete_revenue/total_revenue*100:.2f}%")

Task 3 - Revenue from complete records: $454,498.72
Total revenue: $460,727.86
Percentage: 98.65%


In [54]:
# Show examples of each category
print("Examples from each merge category:")
print("\n--- MATCHED (both) ---")
display(merged_full[merged_full['_merge'] == 'both'][[
    'order_id', 'product_id', 'product', 'revenue'
]].head(3))

print("\n--- SALES ORPHANS (left_only) ---")
display(merged_full[merged_full['_merge'] == 'left_only'][[
    'order_id', 'product_id', 'revenue'
]].head(3))

print("\n--- PRODUCT ORPHANS (right_only) ---")
display(merged_full[merged_full['_merge'] == 'right_only'][[
    'product_id', 'product', 'status'
]])

Examples from each merge category:

--- MATCHED (both) ---


,order_id,product_id,product,revenue
0,SO-10001,P-ADD,Add-on,556.34
1,SO-10005,P-ADD,Add-on,1012.71
2,SO-10007,P-ADD,Add-on,686.96



--- SALES ORPHANS (left_only) ---


,order_id,product_id,revenue
158,SO-10016,P-OEM,555.74
159,SO-10019,P-OEM,363.94
160,SO-10058,P-OEM,206.22



--- PRODUCT ORPHANS (right_only) ---


,product_id,product,status
114,P-BTA,Beta,Preview
157,P-LGC,Legacy,Discontinued


---
# Exercise 8: Region Mapping with Left Join

**Goal:** Add region names to sales using dimension table

In [55]:
# Task 1: Join with regions
sales_regions = sales.merge(regions, on='region_code', how='left')

print(f"Task 1 - Sales with regions: {len(sales_regions)} rows")

Task 1 - Sales with regions: 260 rows


In [56]:
# Task 2: Find unmapped regions
unmapped = sales_regions[sales_regions['region'].isna()]

print(f"Task 2 - Sales with unmapped regions: {len(unmapped)} rows")
if len(unmapped) > 0:
    print("\nUnmapped region codes:")
    display(unmapped['region_code'].value_counts())

Task 2 - Sales with unmapped regions: 10 rows

Unmapped region codes:


region_code
LATAM    10
Name: count, dtype: int64

In [57]:
# Task 3: Clean region column
sales_regions['region_clean'] = sales_regions['region'].fillna('Unknown Region')

print("Task 3 - Region column cleaned (NaN filled with 'Unknown Region')")

Task 3 - Region column cleaned (NaN filled with 'Unknown Region')


In [58]:
# Task 4: Revenue by region
by_region = sales_regions.groupby('region_clean')['revenue'].sum()

print("Task 4 - Revenue by Region:")
display(by_region.sort_values(ascending=False).round(2))

Task 4 - Revenue by Region:


region_clean
AMER             168711.06
EMEA             134487.20
APAC             113627.64
Nordics           29626.41
Unknown Region    14275.55
Name: revenue, dtype: float64

In [59]:
# Task 5: The LATAM/ANZ mystery
in_sales = set(sales['region_code'].unique())
in_dim = set(regions['region_code'].unique())

print(f"Task 5 - Region Code Analysis:")
print(f"Codes in sales but NOT in dimension: {in_sales - in_dim if (in_sales - in_dim) else 'None'}")
print(f"Codes in dimension but NOT in sales: {in_dim - in_sales if (in_dim - in_sales) else 'None'}")

print("\n🔍 The Mystery: LATAM is used in sales but not in the official regions table!")
print("   ANZ is in the regions table but has no sales yet.")

Task 5 - Region Code Analysis:
Codes in sales but NOT in dimension: {'LATAM'}
Codes in dimension but NOT in sales: {'ANZ'}

🔍 The Mystery: LATAM is used in sales but not in the official regions table!
   ANZ is in the regions table but has no sales yet.


---
# Exercise 9: Data Quality Check - Revenue Validation

**Goal:** Verify that revenue = units × unit_price

In [60]:
# Task 1: Calculate expected revenue
sales['expected_revenue'] = sales['units'] * sales['unit_price']

print("Task 1 - Expected revenue calculated")

Task 1 - Expected revenue calculated


In [61]:
# Task 2: Compare with recorded revenue
sales['revenue_diff'] = abs(sales['revenue'] - sales['expected_revenue'])

print("Task 2 - Revenue differences:")
print(f"Max difference: ${sales['revenue_diff'].max():.2f}")
print(f"Mean difference: ${sales['revenue_diff'].mean():.2f}")
print(f"Median difference: ${sales['revenue_diff'].median():.2f}")

Task 2 - Revenue differences:
Max difference: $0.19
Mean difference: $0.04
Median difference: $0.03


In [62]:
# Task 3: Flag discrepancies >$0.01
sales['has_discrepancy'] = sales['revenue_diff'] > 0.01
discrepancy_count = sales['has_discrepancy'].sum()

print(f"Task 3 - Records with discrepancies >$0.01: {discrepancy_count}")

if discrepancy_count > 0:
    print("\nExample discrepancies:")
    display(sales[sales['has_discrepancy']][[
        'order_id', 'units', 'unit_price', 'revenue', 'expected_revenue', 'revenue_diff'
    ]].head())

Task 3 - Records with discrepancies >$0.01: 203

Example discrepancies:


,order_id,units,unit_price,revenue,expected_revenue,revenue_diff
0,SO-10000,6,513.77,3082.59,3082.62,0.03
1,SO-10001,18,30.91,556.34,556.38,0.04
2,SO-10002,8,302.84,2422.68,2422.72,0.04
3,SO-10003,37,112.30,4155.00,4155.10,0.10
4,SO-10004,23,108.11,2486.50,2486.53,0.03


In [63]:
# Task 4: Percentage with discrepancies
discrepancy_pct = (discrepancy_count / len(sales)) * 100

print(f"Task 4 - Percentage with discrepancies: {discrepancy_pct:.1f}%")

Task 4 - Percentage with discrepancies: 78.1%


In [64]:
# Task 5: Discrepancies by product and region
if discrepancy_count > 0:
    by_product = (sales.groupby('product_id')['has_discrepancy'].mean() * 100).round(1)
    by_region = (sales.groupby('region_code')['has_discrepancy'].mean() * 100).round(1)
    
    print("Task 5 - Discrepancy rate by product:")
    display(by_product[by_product > 0])
    
    print("\nDiscrepancy rate by region:")
    display(by_region[by_region > 0])
else:
    print("Task 5 - No discrepancies found!")

Task 5 - Discrepancy rate by product:


product_id
P-ADD   89.70
P-ANL   78.80
P-BSC   88.50
P-ENT   47.60
P-OEM   55.60
P-PLS   91.70
P-PRO   78.90
P-TRI   66.70
Name: has_discrepancy, dtype: float64


Discrepancy rate by region:


region_code
AMER    84.60
APAC    76.70
EMEA    77.60
LATAM   70.00
NORD    65.20
Name: has_discrepancy, dtype: float64

---
# Exercise 10: String Cleaning for Better Joins

**Goal:** Handle common string matching issues

In [65]:
# Task 1: Check for whitespace issues
def check_whitespace(df, col):
    has_leading = df[col].str.match(r'^\s').any()
    has_trailing = df[col].str.match(r'\s$').any()
    return has_leading or has_trailing

print("Task 1 - Checking for whitespace issues:")
for df, name in [(sales, 'sales'), (products, 'products'), (regions, 'regions')]:
    for col in df.select_dtypes(include='object').columns:
        if col not in ['order_date', 'order_id', 'year_month']:
            has_ws = check_whitespace(df, col)
            status = "⚠️  HAS WHITESPACE" if has_ws else "✓ Clean"
            print(f"{status}: {name}.{col}")

Task 1 - Checking for whitespace issues:
✓ Clean: sales.product_id
✓ Clean: sales.region_code
✓ Clean: sales.weekday
✓ Clean: products.product_id
✓ Clean: products.product
✓ Clean: products.category
✓ Clean: products.status
✓ Clean: regions.region_code
✓ Clean: regions.region


In [66]:
# Task 2: Check for case inconsistencies
print("Task 2 - Checking for case inconsistencies:")
print(f"Sales product_id unique values: {sales['product_id'].nunique()}")
print(f"Sales product_id unique (case-insensitive): {sales['product_id'].str.upper().nunique()}")
print("\n✓ No case inconsistencies found")

Task 2 - Checking for case inconsistencies:
Sales product_id unique values: 8
Sales product_id unique (case-insensitive): 8

✓ No case inconsistencies found


In [67]:
# Task 3: Create cleaned versions
def clean_string_column(series):
    """Clean a string column for joining"""
    if series.dtype == 'object':
        return (series
                .str.strip()           # Remove whitespace
                .str.upper()           # Standardize case
                .str.replace(r'\s+', ' ', regex=True))  # Collapse spaces
    return series

# Apply cleaning
sales_clean = sales.copy()
products_clean = products.copy()
regions_clean = regions.copy()

for df in [sales_clean, products_clean, regions_clean]:
    for col in df.select_dtypes(include='object').columns:
        if col not in ['order_date', 'order_id', 'year_month', 'weekday']:
            df[col] = clean_string_column(df[col])

print("Task 3 - Cleaned versions created")

Task 3 - Cleaned versions created


In [68]:
# Task 4: Compare joins
original_inner = sales.merge(products, on='product_id', how='inner')
cleaned_inner = sales_clean.merge(products_clean, on='product_id', how='inner')

print(f"Task 4 - Join comparison:")
print(f"Original inner join: {len(original_inner)} rows")
print(f"Cleaned inner join: {len(cleaned_inner)} rows")
print(f"Difference: {len(cleaned_inner) - len(original_inner)} rows")
print("\n✓ No difference because data is already clean!")

Task 4 - Join comparison:
Original inner join: 242 rows
Cleaned inner join: 242 rows
Difference: 0 rows

✓ No difference because data is already clean!


In [69]:
# Task 5: Show the cleaning function
print("Task 5 - Reusable cleaning function:")
print("""\ndef clean_string_column(series):
    if series.dtype == 'object':
        return (series
                .str.strip()
                .str.upper()
                .str.replace(r'\\s+', ' ', regex=True))
    return series

# Use before joining:
df['product_id'] = clean_string_column(df['product_id'])
""")
print("\n💡 Always clean strings before joining - it's a best practice!")

Task 5 - Reusable cleaning function:

def clean_string_column(series):
    if series.dtype == 'object':
        return (series
                .str.strip()
                .str.upper()
                .str.replace(r'\s+', ' ', regex=True))
    return series

# Use before joining:
df['product_id'] = clean_string_column(df['product_id'])


💡 Always clean strings before joining - it's a best practice!


---
# Exercise 11: Multi-Table Join Chain

**Goal:** Combine all three tables in sequence

In [70]:
# Tasks 1-4: Chain all joins
comprehensive = (
    sales
    .merge(products, on='product_id', how='left')
    .merge(regions, on='region_code', how='left')
)

print(f"Comprehensive view: {len(comprehensive)} rows × {len(comprehensive.columns)} columns")
print(f"\nAll {len(sales)} sales retained with full product and region details!")
print(f"\nColumns: {comprehensive.columns.tolist()}")

Comprehensive view: 260 rows × 22 columns

All 260 sales retained with full product and region details!

Columns: ['order_id', 'order_date', 'product_id', 'region_code', 'units', 'unit_price', 'revenue', 'year_month', 'year', 'month', 'quarter', 'weekday', 'fiscal_quarter', 'days_since', 'expected_revenue', 'revenue_diff', 'has_discrepancy', 'product', 'category', 'list_price', 'status', 'region']


In [71]:
# Task 5: Revenue by category and region
cat_region = comprehensive.pivot_table(
    index='category',
    columns='region',
    values='revenue',
    aggfunc='sum',
    fill_value=0
).round(2)

print("Task 5 - Revenue by Category × Region:")
display(cat_region)

print("\nTotal by category:")
display(comprehensive.groupby('category')['revenue'].sum().sort_values(ascending=False).round(2))

Task 5 - Revenue by Category × Region:


region,AMER,APAC,EMEA,Nordics
category,,,,
Addon,36303.44,22522.15,38632.28,5685.37
Core,83139.57,59097.14,79625.67,17768.50
Suite,48927.76,29289.86,13617.57,5945.77



Total by category:


category
Core    244595.41
Addon   106504.97
Suite   103398.34
Name: revenue, dtype: float64

---
# Exercise 12: Aggregation After Joins (Advanced)

**Goal:** Answer complex business questions using joined data

In [72]:
# Task 1: Revenue by category
task1 = comprehensive.groupby('category')['revenue'].sum().sort_values(ascending=False)

print("Task 1 - Total Revenue by Category:")
display(task1.round(2))

Task 1 - Total Revenue by Category:


category
Core    244595.41
Addon   106504.97
Suite   103398.34
Name: revenue, dtype: float64

In [73]:
# Task 2: Average discount by region
comprehensive['discount_rate'] = (
    (comprehensive['list_price'] - comprehensive['unit_price']) 
    / comprehensive['list_price']
)
task2 = comprehensive.groupby('region')['discount_rate'].mean() * 100

print("Task 2 - Average Discount Rate by Region (%):")
display(task2.round(2))

Task 2 - Average Discount Rate by Region (%):


region
AMER       0.49
APAC      -0.75
EMEA      -0.05
Nordics   -0.01
Name: discount_rate, dtype: float64

In [74]:
# Task 3: Average order size by category
task3 = comprehensive.groupby('category')['units'].mean()

print("Task 3 - Average Order Size (units) by Category:")
display(task3.round(2))
print("\n💡 Suite products (Enterprise) have much smaller order sizes!")

Task 3 - Average Order Size (units) by Category:


category
Addon   19.32
Core    19.24
Suite    5.05
Name: units, dtype: float64


💡 Suite products (Enterprise) have much smaller order sizes!


In [75]:
# Task 4: Revenue by status
task4 = comprehensive.groupby('status')['revenue'].sum().sort_values(ascending=False)

print("Task 4 - Revenue by Product Status:")
display(task4.round(2))
print("\n(Only Active products have sales)")

Task 4 - Revenue by Product Status:


status
Active   454498.72
Name: revenue, dtype: float64


(Only Active products have sales)


In [76]:
# Task 5: Peak month by category
comprehensive['month_period'] = comprehensive['order_date'].dt.to_period('M')
monthly_cat = comprehensive.groupby(['category', 'month_period'])['revenue'].sum()
peak_months = monthly_cat.groupby(level=0).idxmax()

print("Task 5 - Peak Revenue Month by Category:")
for category, (cat, month) in peak_months.items():
    peak_rev = monthly_cat[cat, month]
    print(f"{category}: {month} (${peak_rev:,.2f})")

Task 5 - Peak Revenue Month by Category:
Addon: 2024-01 ($17,976.25)
Core: 2025-02 ($18,353.55)
Suite: 2024-09 ($15,190.15)


---
# Exercise 13: Pivot Tables & Cross-Tabs

**Goal:** Create Excel-style pivot tables

In [77]:
# Task 1: Product × Region pivot
pivot_rev = comprehensive.pivot_table(
    index='product',
    columns='region',
    values='revenue',
    aggfunc='sum',
    fill_value=0
).round(2)

print("Task 1 - Product × Region Revenue Matrix:")
display(pivot_rev)

Task 1 - Product × Region Revenue Matrix:


region,AMER,APAC,EMEA,Nordics
product,,,,
Add-on,7157.68,6028.36,13550.26,1122.64
Analytics,29145.76,16493.79,25082.02,4562.73
Basic,23934.16,17957.99,13510.18,5719.80
Enterprise,48927.76,29289.86,13617.57,5945.77
Plus,33820.45,25607.96,23151.35,3828.26
Pro,25384.96,15531.19,42964.14,8220.44


In [78]:
# Task 2: Month × Product units
pivot_units = comprehensive.pivot_table(
    index='month_period',
    columns='product',
    values='units',
    aggfunc='sum',
    fill_value=0
)

print("Task 2 - Month × Product Units Matrix (first 10 months):")
display(pivot_units.head(10))

Task 2 - Month × Product Units Matrix (first 10 months):


product,Add-on,Analytics,Basic,Enterprise,Plus,Pro
month_period,,,,,,
2024-01,32,57,49,0,34,10
2024-02,33,6,131,7,14,13
2024-03,29,0,104,0,73,0
2024-04,53,21,91,21,74,29
2024-05,0,22,102,17,30,19
2024-06,34,12,93,18,35,8
2024-07,30,4,63,2,71,28
2024-08,0,36,57,11,25,6
2024-09,72,0,39,31,58,37


In [79]:
# Task 3: Percentage within each region
pivot_pct = pivot_rev.div(pivot_rev.sum(axis=0), axis=1) * 100

print("Task 3 - Product Revenue % within Each Region:")
display(pivot_pct.round(1))

Task 3 - Product Revenue % within Each Region:


region,AMER,APAC,EMEA,Nordics
product,,,,
Add-on,4.30,5.40,10.30,3.80
Analytics,17.30,14.90,19.00,15.50
Basic,14.20,16.20,10.20,19.50
Enterprise,29.10,26.40,10.30,20.20
Plus,20.10,23.10,17.60,13.00
Pro,15.10,14.00,32.60,28.00


In [80]:
# Task 4: Highest average order value
avg_order = comprehensive.groupby(['product', 'region'])['revenue'].mean()
max_combo = avg_order.idxmax()
max_value = avg_order.max()

print(f"Task 4 - Highest Average Order Value:")
print(f"Product-Region: {max_combo}")
print(f"Average Order Value: ${max_value:,.2f}")

Task 4 - Highest Average Order Value:
Product-Region: ('Enterprise', 'APAC')
Average Order Value: $2,928.99


---
# Exercise 14: Unpivot (Melt) for Reshaping

**Goal:** Convert wide format back to long format

In [81]:
# Tasks 1-2: Melt the pivot table
long_format = pivot_rev.reset_index().melt(
    id_vars='product',
    var_name='region',
    value_name='revenue'
)

print(f"Tasks 1-2 - Unpivoted data: {len(long_format)} rows")
print("\nSample of long format:")
display(long_format.head(10))

Tasks 1-2 - Unpivoted data: 24 rows

Sample of long format:


,product,region,revenue
0,Add-on,AMER,7157.68
1,Analytics,AMER,29145.76
2,Basic,AMER,23934.16
3,Enterprise,AMER,48927.76
4,Plus,AMER,33820.45
5,Pro,AMER,25384.96
6,Add-on,APAC,6028.36
7,Analytics,APAC,16493.79
8,Basic,APAC,17957.99
9,Enterprise,APAC,29289.86


In [82]:
# Task 3: Verify totals
original_total = comprehensive['revenue'].sum()
melted_total = long_format['revenue'].sum()

print(f"Task 3 - Total verification:")
print(f"Original total: ${original_total:,.2f}")
print(f"After pivot/melt: ${melted_total:,.2f}")
print(f"Difference: ${abs(original_total - melted_total):,.2f}")
print(f"\nMatch: {abs(original_total - melted_total) < 1.0}")
print("\n(Difference is from orphan products/regions excluded in pivot)")

Task 3 - Total verification:
Original total: $460,727.86
After pivot/melt: $440,555.08
Difference: $20,172.78

Match: False

(Difference is from orphan products/regions excluded in pivot)


In [83]:
# Task 4: Non-zero combinations
long_nonzero = long_format[long_format['revenue'] > 0]

print(f"Task 4 - Non-zero combinations: {len(long_nonzero)} out of {len(long_format)}")

Task 4 - Non-zero combinations: 24 out of 24


---
# Exercise 15: Finding Duplicates & Near-Duplicates

**Goal:** Identify and handle duplicate records

In [84]:
# Task 1: Exact duplicates
exact_dupes = sales.duplicated()

print(f"Task 1 - Exact duplicates: {exact_dupes.sum()}")

Task 1 - Exact duplicates: 0


In [85]:
# Task 2: Duplicates on key columns
key_dupes = sales.duplicated(subset=['order_id', 'product_id'])

print(f"Task 2 - Duplicates on order_id + product_id: {key_dupes.sum()}")

Task 2 - Duplicates on order_id + product_id: 0


In [86]:
# Task 3: Near-duplicates
near_dupes = sales.groupby(['order_id', 'product_id']).filter(lambda x: len(x) > 1)

print(f"Task 3 - Near-duplicates (same key, different values): {len(near_dupes)}")

Task 3 - Near-duplicates (same key, different values): 0


In [87]:
# Task 4: Strategies for handling duplicates
print("Task 4 - Duplicate handling strategies:\n")

print("Strategy 1 - Keep first occurrence:")
deduped1 = sales.drop_duplicates(subset=['order_id', 'product_id'], keep='first')
print(f"  Rows after deduplication: {len(deduped1)}")

print("\nStrategy 2 - Keep highest revenue:")
deduped2 = (sales
    .sort_values('revenue', ascending=False)
    .drop_duplicates(subset=['order_id', 'product_id'], keep='first')
)
print(f"  Rows after deduplication: {len(deduped2)}")

print("\nStrategy 3 - Flag for manual review:")
sales['is_duplicate'] = sales.duplicated(subset=['order_id', 'product_id'], keep=False)
print(f"  Records flagged for review: {sales['is_duplicate'].sum()}")

print("\n✓ Good news: This dataset has no duplicates!")

Task 4 - Duplicate handling strategies:

Strategy 1 - Keep first occurrence:
  Rows after deduplication: 260

Strategy 2 - Keep highest revenue:
  Rows after deduplication: 260

Strategy 3 - Flag for manual review:
  Records flagged for review: 0

✓ Good news: This dataset has no duplicates!


---
# BONUS: Reusable Data Cleaning Pipeline

**Goal:** Build a function that handles all cleaning and joining

In [88]:
def prepare_sales_data(sales_path, products_path, regions_path):
    """
    Load and prepare sales data for analysis
    
    Args:
        sales_path: Path to sales CSV
        products_path: Path to products CSV
        regions_path: Path to regions CSV
    
    Returns:
        pd.DataFrame: Clean, joined, analysis-ready data
    """
    # Load
    sales = pd.read_csv(sales_path)
    products = pd.read_csv(products_path)
    regions = pd.read_csv(regions_path)
    
    # Clean string columns
    for df in [sales, products, regions]:
        for col in df.select_dtypes(include='object').columns:
            if col not in ['order_date', 'order_id']:
                df[col] = df[col].str.strip().str.upper()
    
    # Parse dates
    sales['order_date'] = pd.to_datetime(sales['order_date'])
    
    # Perform joins
    result = (sales
        .merge(products, on='product_id', how='left')
        .merge(regions, on='region_code', how='left')
    )
    
    # Add calculated fields
    result['discount_pct'] = (
        (result['list_price'] - result['unit_price']) / result['list_price'] * 100
    )
    result['is_orphan'] = result['product'].isna()
    result['expected_revenue'] = result['units'] * result['unit_price']
    result['revenue_valid'] = abs(result['revenue'] - result['expected_revenue']) < 0.01
    
    # Add time dimensions
    result['year'] = result['order_date'].dt.year
    result['quarter'] = result['order_date'].dt.quarter
    result['month'] = result['order_date'].dt.month
    result['year_month'] = result['order_date'].dt.to_period('M')
    
    return result

print("✓ Pipeline function defined")

✓ Pipeline function defined


In [91]:
# Use the pipeline
clean_data = prepare_sales_data(
    'files/sales_records.csv',
    'files/products_master.csv',
    'files/regions_dim.csv'
)

print(f"✓ Pipeline executed successfully!")
print(f"Clean dataset: {len(clean_data)} rows × {len(clean_data.columns)} columns")

print(f"\nData quality summary:")
print(f"- Orphan records: {clean_data['is_orphan'].sum()}")
print(f"- Revenue validation issues: {(~clean_data['revenue_valid']).sum()}")
print(f"- Average discount: {clean_data['discount_pct'].mean():.2f}%")

✓ Pipeline executed successfully!
Clean dataset: 260 rows × 20 columns

Data quality summary:
- Orphan records: 18
- Revenue validation issues: 203
- Average discount: -0.12%
